# Background estimation

In this notebook we'll perform background estimation. We shall make use of the so called **ABCD method**. 
We describe the method briefly in what follows before applying it to our analysis.
Let's assume we identify two variables ($v_1$ and $v_2$) which have uncorrelated distributions for the background.
A cut each of these variables will have an efficiency which is indepedent of whatever selections have been performed on the other variable.
Let's say we divide the 2D-plane ($v_1-v_2$) in four regions (A,B,C,D) being A is the final signal region where both selection requirements on $v_1$ and $v_2$ are applied. Such plane is represented in the following figure.

![Representation of the 2D-plane used to define the ABCD regions for background estimation](img/abcd_method.png "ABCD method")

If the selections are indeed uncorrelated for the background, then one can assume that the ratio of events in each regions can be related as:

\begin{equation}
\frac{N_{A}^{bkg}}{N_{B}^{bkg}} =\frac{N_{C}^{bkg}}{N_{D}^{bkg}} \Leftrightarrow N_{A}^{bkg} =N_{B}^{bkg}\cdot\frac{N_{C}^{bkg}}{N_{D}^{bkg}}
\end{equation}

In the notebook below we shall apply this method to our analysis using two uncorrelated variables: the number of extra PF candidates and the $\Delta\xi$.

## Loading the data and simulations to test the method

The procedure used here is analogous to the one used in the Classification-Training notebook.
After loading the data we run a prepare data method such that we can also run the classifier we have previously trained. In the prepare data method we'll already include the output of the classifier.

In [ ]:
from snippets.GetData import *

proton_selection = "MultiRP"
PATH='/eos/user/c/cmsdas/long-exercises/pps-exclusive-dilepton/h5py'
#PATH='output'
#eras=['B','C','D','E','F'] #uncomment to use all data
eras=['B']
stream='El' # 'El' OR 'Mu'


print('[Signal simulation]')
df_signal,df_counts_signal = GetData(PATH+'/output-GGTo{}_Elastic_v0_signal_xa120_era2017_preTS2.h5'.format('EE' if stream=='El' else 'MuMu'))
print('Selection counts')
print(df_counts_signal)

print('\n')
print('[Data (to be used as background)]')
data_files = [PATH+'/output-UL2017{}-{}-Rand20.h5'.format(era,stream) for era in eras]
df_bkg,df_counts_bkg = GetData(data_files,chunk_size=1000000)
print('Selection counts')
print(df_counts_bkg)
print('\n')

print('[Data]')
data_files = [PATH+'/output-UL2017{}-{}.h5'.format(era,stream) for era in eras]
df_data,df_counts_data = GetData(data_files,chunk_size=1000000)
print('Selection counts')
print(df_counts_data)

In [ ]:
def PrepareData(df,clf=None,train_vars=['Lep0Pt', 'Lep1Pt', 'InvMass',   'XiMuMu', 'Xi', 'Acopl','ExtraPfCands_v1']):
    
    """applies baseline selection cuts and adds the Xi resolution and the classifier prediction"""

    msk = ( df["InvMass"] >= 110. )

    msk1 = None
    msk2 = None
    if proton_selection == "SingleRP":
        # Single-RP in pixel stations
        msk1_arm = ( df["RPId1"] == 23 )
        msk2_arm = ( df["RPId1"] == 123 )
        multiRP=0
    elif proton_selection == "MultiRP":
        # Multi-RP
        msk1_arm = ( df["Arm"] == 0 )
        msk2_arm = ( df["Arm"] == 1 )
        multiRP=1
   
    #assign the xi from the dileptons based on the proton side
    df[ "XiMuMu" ] = np.nan
    df[ "XiMuMu" ].where( ~msk1_arm, df[ "XiMuMuPlus" ],  inplace=True )
    df[ "XiMuMu" ].where( ~msk2_arm, df[ "XiMuMuMinus" ], inplace=True )

    #add a resolution column
    df[ 'DeltaXi'] = df[ "XiMuMu" ]-df[ "Xi" ]
    df[ 'AbsDeltaXi'] = np.abs(df['DeltaXi'])
    
    #filter the data for reconstructed protons
    msk1 = msk & ( df["MultiRP"] == multiRP) & msk1_arm
    msk2 = msk & ( df["MultiRP"] == multiRP) & msk2_arm   
    df=df[msk1 | msk2].copy()

    #add the result of the classifier prediction
    if clf and len(train_vars)>0:
        X=df[train_vars]
        df['clf_proba'] = clf.predict_proba(X)[:,1]
        df['clf_categ'] = clf.predict(X).astype(int)
        
    return df


#load the trained model
from joblib import load
clf = load('pps_longexercise_clf.joblib') 

df_signal_prep=PrepareData(df_signal,clf)
df_bkg_prep=PrepareData(df_bkg,clf)
df_data_prep=PrepareData(df_data,clf)

## Running a simple ABCD prediction

In the following cell we plot the ABCD regions for signal, data and background in each one of them.
For the data we select only the events which are categorized as background events (i.e. blind the final signal selection).
What can you conclude from the following plots?

**TASK 1**

Compute the correlation of the two variables for each of the 3 datasets being plotted.

In [ ]:
import matplotlib.pyplot as plt

v1='AbsDeltaXi'
cut_v1=0.01
range_v1=(0,0.15)

v2='ExtraPfCands_v1'
cut_v2=6
range_v2=(0,60)

def doScatter(df,label, ax):
    ax.hist2d(df[v1],df[v2],cmap='Blues')
    ax.set_xlim(*range_v1)
    ax.set_ylim(*range_v2)
    ax.set_xlabel(v1)
    ax.set_ylabel(v2)
    ax.plot([cut_v1,cut_v1],[range_v2[0],range_v2[1]],color='lightgray')
    ax.plot([range_v1[0],range_v1[1]],[cut_v2,cut_v2],color='lightgray')
    ax.text(0.2,0.9,label, transform=ax.transAxes)
    ax.text(cut_v1*0.7,cut_v2*0.7,'A')
    ax.text(cut_v1*1.2,cut_v2*0.7,'B')
    ax.text(cut_v1*0.7,cut_v2*1.2,'C')
    ax.text(cut_v1*1.2,cut_v2*1.2,'D')
    

    
fig, ax = plt.subplots(1, 3, figsize=(15,5))
doScatter(df_signal_prep,'Signal',ax[0])
doScatter(df_bkg_prep,'Background',ax[1])
doScatter(df_data_prep[df_data_prep['clf_categ']==0],'Data (background category)',ax[2])

plt.show()

We now apply the ABCD method with a simple function... and the background is estimated!

**TASK 2**

Discusion: Is this method perfect? 
How is the closure affected if the cuts are different?
Is there any extra dependency (e.g. on data-taking era, pileup, dilepton variables, etc.)?
What are the final uncertainties we can associate to this method?

In [ ]:
def getABCDYields(df):
    
    """gets the yields in the different abcd regions"""

    mask_a=(df[v1]<cut_v1) &(df[v2]<cut_v2)
    mask_b=(df[v1]>cut_v1) &(df[v2]<cut_v2)
    mask_c=(df[v1]<cut_v1) &(df[v2]>cut_v2)
    mask_d=(df[v1]>cut_v1) &(df[v2]>cut_v2)
    
    yields={'A':df[mask_a].shape[0],
            'B':df[mask_b].shape[0],
            'C':df[mask_c].shape[0],
            'D':df[mask_d].shape[0]}

    return yields
 
def getABCDPrediction(df):
    
    """
    starting from the yields in the different regions, applies the ABCD formula
    returns the number of events observed in the signal region and the estimated number of events
    """
    
    yields=getABCDYields(df)
    
    return yields['A'],yields['B']*yields['C']/yields['D']
    

print('\nData (background category)')
nA,nAest=getABCDPrediction(df_data_prep[df_data_prep['clf_categ']==0])
print('nA(obs)=%d nA(est)=%3.1f bias=%3.3f'%(nA,nAest,nAest/nA))

print('\nData (background)')
nA,nAest=getABCDPrediction(df_bkg_prep)
print('nA(obs)=%d nA(est)=%3.1f bias=%3.3f'%(nA,nAest,nAest/nA))

## Gearing up for the statistical analysis

In the following we'll prepare a so called datacard reporting the yields in each region for the signal and the background, as well as their associated uncertainties. This datacard will be used as input for the Higgs Combination tool. More details on this tool can be found in [here](https://cms-analysis.github.io/HiggsAnalysis-CombinedLimit).

We start by preparing a ROOT file which will contain the counting histograms.
For the moment this will be simple 1bin histograms with event counts but after you understand the example you can evolve this to be based on the distribution of a variable.

In [ ]:
import ROOT

samples={'bkg':df_bkg_prep,
         'sig':df_signal_prep,
         'data_obs':df_data_prep}

#loop over the samples
#get yields in the different regions
yields_abcd={}
for df_name,df in samples.items():    
    yields_abcd[df_name]=getABCDYields(df)
    
#apply a scale factor to the background estimation such that it matches the data in region D
bkgSF=yields_abcd['data_obs']['D']/yields_abcd['bkg']['D']
for x in yields_abcd['bkg']: 
    yields_abcd['bkg'][x]=bkgSF*yields_abcd['bkg'][x]
print('Background estimation has been scaled by %3.3f'%bkgSF)

#template for the shape of the distribution to analyze
shapeH=ROOT.TH1F('shape','shape',1,0,1)

#open the output file
fOut=ROOT.TFile.Open('shapes.root','RECREATE')

#loop over the samples
for df_name in samples:    
    
    #save the counts in histogram and store them in the file
    for region in yields:
        shape=shapeH.Clone('{}_{}'.format(df_name,region))
        shape.SetBinContent(1,yields_abcd[df_name][region])
        shape.SetDirectory(fOut)
        shape.Write()

fOut.Close()

In [ ]:
#open a txt file to hold the datacard
dc=open('datacard.dat','w')

dc.write('imax *  number of channels\n')
dc.write('jmax *  number of processes -1\n')
dc.write('kmax *  number of nuisance parameters (sources of systematical uncertainties)\n')
dc.write('-------\n')

#spell out out the shapes are stored in the ROOT file
dc.write('shapes * * shapes.root $PROCESS_$CHANNEL $PROCESS_$CHANNEL_$SYSTEMATIC\n')

#add the data observations
dc.write('bin          A   B   C   D\n')
dc.write('observation  {0} {1} {2} {3}\n'.format(yields_abcd['data_obs']['A'],
                                                 yields_abcd['data_obs']['B'],
                                                 yields_abcd['data_obs']['C'],
                                                 yields_abcd['data_obs']['D']))
dc.write('-------\n')

#add the expectations for signal and background
dc.write('bin         A    A    B   B   C   C   D   D\n')
dc.write('process     sig  bkg  sig bkg sig bkg sig bkg\n')
dc.write('process     0    1    0   1   0   1   0   1\n')
dc.write('rate        {0:.3f}  {1:.3f}  {2:.3f} {3:.3f} {4:.3f} {5:.3f} {6:.3f} {7:.3f}\n'.format(yields_abcd['sig']['A'],
                                                                                                  yields_abcd['bkg']['A'],
                                                                                                  yields_abcd['sig']['B'],
                                                                                                  yields_abcd['bkg']['B'],
                                                                                                  yields_abcd['sig']['C'],
                                                                                                  yields_abcd['bkg']['C'],
                                                                                                  yields_abcd['sig']['D'],
                                                                                                  yields_abcd['bkg']['D'],
                                                                                                 ))
dc.write('-------\n')

#spell-out the ABCD method
dc.write('alpha rateParam A bkg (@0*@1/@2) beta,gamma,delta\n')
dc.write('beta  rateParam B bkg {}\n'.format(yields_abcd['bkg']['B']))
dc.write('gamma rateParam C bkg {}\n'.format(yields_abcd['bkg']['C']))
dc.write('delta rateParam D bkg {}\n'.format(yields_abcd['bkg']['D']))
         
#all done
dc.close()

print('datacard.dat is ready to be used with combine\n\n')
!cat datacard.dat

## Running the Higgs Combination tool for the statistical analysis

The short exercise on statistical analysis should be used as a reference - [link](https://cmsdas.github.io/statistics-short-exercise/).
as well as the documentation of combine ([here](https://cms-analysis.github.io/HiggsAnalysis-CombinedLimit)). 
Some basic commands are listed below just to get started:
    
* Create the workspace: `text2workspace.py datacard.dat -o workspace.root`
* Running fit diagnostics: `combine -M FitDiagnostics workspace.root -t -1 --expectSignal=1`
* Compute the expected significance (asymptotic): `combine workspace.root -M Significance -t -1 --expectSignal=1`
* Likelihood scan: `combine workspace.root -M MultiDimFit --algo grid --points 100 --fastScan --expectSignal=1 -t -1`
* Compute the goodness-of-fit: `combine -M GoodnessOfFit workspace.root --algo=saturated --expectSignal=1 -t -1`